In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from scipy import optimize
import time
import os
import os.path
import zipfile
import pandas as pd
from scipy.optimize import curve_fit, least_squares
import sys
import gmsh
import math

import matplotlib.image as mpimg
import re

import pyvista as pv
import ipywidgets as widgets
pv.set_plot_theme("document")

plt.rcParams['figure.figsize'] = [12, 9]
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.family'] = "Serif"
plt.rcParams['font.size'] = 15

from pyvirtualdisplay import Display
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()
    
user_name=!whoami # get user name
user_name=user_name[0]
um_view = "$HOME/um_view"

# new executables
mixed_exe = um_view + "/*eshelbian_*/ep"
standard_exe = um_view + "/tutorials/vec-2/nonlinear_elastic"


In [ ]:
def generate_config(params):
    # Open the file for writing
    with open(params.config_name, 'w') as f:
        # FIX_ALL boundary condition (do not change)
        data = ['[SET_ATTR_PRESSURE_UNIFORM_OUT]', 'number_of_attributes=1', 'user1='+str(params.p_out)]
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        data = ['[SET_ATTR_PRESSURE_UNIFORM_IN]', 'number_of_attributes=1', 'user1='+str(params.p_in)]
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data
            print(line)
            

In [ ]:
class AttrDict(dict):
    def __getattr__(self, attr):
        if attr in self:
            return self[attr]
        raise AttributeError(f"'AttrDict' object has no attribute '{attr}'")
        
def generate_mesh_square_circle(params):
    
    # Initialize gmsh
    gmsh.initialize()
    gmsh.option.setNumber("General.Verbosity", 3)
    
    tol = 0.001

    square1 = gmsh.model.occ.add_rectangle(-params.square_length/2, -params.square_length/2, 0, params.square_length, params.square_length)
    circle1 = gmsh.model.occ.add_disk(0, 0, 0, params.circle_radius, params.circle_radius)
    
    gmsh.model.occ.cut([(2,square1)], [(2,circle1)])
    

    # Create the relevant Gmsh data structures from Gmsh model.
    gmsh.model.occ.synchronize()

    gmsh.option.setNumber("Mesh.MeshSizeMin", params.element_size)
    gmsh.option.setNumber("Mesh.MeshSizeMax", params.element_size)
    
    for n in (gmsh.model.getEntities(1)):
        print(n)

    # gmsh.model.addPhysicalGroup(dimention, [number of element], name="name")
    gmsh.model.addPhysicalGroup(2, [1], name="domain")
    # gmsh.model.addPhysicalGroup(1, [1, 2, 3, 4], name="PRESSURE_UNIFORM_OUT")
    gmsh.model.addPhysicalGroup(1, [6,7,8,9], name="PRESSURE_UNIFORM_OUT")
    gmsh.model.addPhysicalGroup(1, [5], name="PRESSURE_UNIFORM_IN")

    # generate a 2D mesh
    gmsh.model.mesh.generate(2)
    
    # save as a .med file
    med_file = params.mesh_file + ".med"
    gmsh.write(med_file)
    
    # close gmsh
    gmsh.finalize()
    
    # translate .med file to a format readable by MoFEM and assign values to physical groups
    h5m_file=params.mesh_file + "_" + str(params.element_size) + ".h5m"    
    !read_med -med_file {med_file} -output_file {h5m_file} -meshsets_config {params.config_name} -dim 2 -adj_dim 1
    
    # visualise the mesh
    if params.show_mesh:
        vtk_file=params.mesh_file + ".vtk"
        !mbconvert {h5m_file} {vtk_file}

        mesh = pv.read(vtk_file)
        mesh = mesh.shrink(0.98)

        p = pv.Plotter(notebook=True)
        p.add_mesh(mesh, smooth_shading=False)

        p.camera_position = "xy"
        p.show(jupyter_backend='ipygany')
        

In [ ]:
params = AttrDict() # Attribute dictionary for storing the parameters

# Pre-processing parameters
params.mesh_file = "square_circle"
params.element_size = 0.2 # element size in the regular mesh
params.show_mesh = True

# boundary condition configuration
params.config_name = "bc.cfg"

# solution parameters
params.log_file = "log" # log file name 
params.nproc = 2
params.order = 2 # approximation order for displacements

In [ ]:
params.square_length = 50
params.circle_radius = 15
params.p_out = 5
params.p_in = 30

In [ ]:
generate_config(params)

In [ ]:
params.show_mesh = True
params.ele_sizes = [1]
for h in params.ele_sizes:
    params.element_size = h
    generate_mesh_square_circle(params)